In [1]:
import gymnasium as gym
from gym import spaces
import numpy as np
import rospy
from geometry_msgs.msg import Pose, PoseStamped, Point, Quaternion
from geometry_msgs.msg import Twist
from visualization_msgs.msg import Marker
from std_srvs.srv import Empty
from nav_msgs.msg import Odometry
import time

In [2]:
# Open AI gym code
class GazeboEnv(gym.Env):
    def __init__(self) -> None:
        super().__init__()
        linear_velocities = np.linspace(-1, 1, 5)
        angular_velocities = np.linspace(-1, 1, 5)
        self.velocity_pairs = [(linear, angular) for linear in linear_velocities for angular in angular_velocities]

        self.action_space = spaces.Discrete(len(self.velocity_pairs)) 
        self.observation_space = spaces.Box(low = np.array([-1, -1]), high = np.array([1, 1]), dtype = np.float32)

    def step(self, action):

        linear, angular = self.velocity_pairs[action - 1]

        rospy.init_node('custom_gym', anonymous=True)
        velocity_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

        pos_x, pos_y, linear_velocity, angular_velocity = None, None, None, None

        def odom_callback(data):
            nonlocal pos_x, pos_y, linear_velocity, angular_velocity

            linear_velocity = data.twist.twist.linear.x
            angular_velocity = data.twist.twist.angular.z

            pos_x = data.pose.pose.position.x
            pos_y = data.pose.pose.position.y

            # rospy.loginfo("Linear Velocity: %f", linear_velocity)
            # rospy.loginfo("Angular Velocity: %f", angular_velocity)

        odom_subscriber = rospy.Subscriber('/odom', Odometry, odom_callback)

        rate = rospy.Rate(10)
        start_time = rospy.get_time()
        odom_subscriber = rospy.Subscriber('/odom', Odometry, odom_callback)

        while (rospy.Time.now() - rospy.Time.from_sec(start_time)).to_sec() < 0.01:
            velocity_command = Twist()
            velocity_command.linear.x = linear
            velocity_command.angular.z = angular

            velocity_publisher.publish(velocity_command)
            rate.sleep()

        # print("Final Position: ({}, {})".format(pos_x, pos_y))
        # print("Final Linear Velocity: {}".format(linear_velocity))
        # print("Final Angular Velocity: {}".format(angular_velocity))

        #Checking for rewards
        self.goal_x = 1
        self.goal_y = 1

        distance_from_goal = np.sqrt((self.goal_x - pos_x)**2 + (self.goal_y - pos_y)**2)
    




        #Checking if episode has ended or not
        done = False
        if distance_from_goal < 0.5:
            done = True
        
        #If it gets lost
        if distance_from_goal > 2:
            done = True


        return np.array([linear_velocity, angular_velocity]), 0, done, {}

    def reset(self):
        try:
            reset = rospy.ServiceProxy("/gazebo/reset_world", Empty)
            reset()
        except rospy.ServiceException as e:
            print(f"{type(e)}: {e}")
    
    def render(self):
        pass
    

In [3]:
env = GazeboEnv()

obs = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    if(done):
        env.reset()
        done = False
        
    env.render()

print("Ended")

# count = 0
# while (count < 10):
#     count += 1
#     action = env.action_space.sample()
#     obs, reward, done, info = env.step(action)
#     env.render()

/home/voyagerani/.local/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
